Let's just get a quick sparsity overview of the methods so far.

In [14]:
import torch
from torch.utils.data import DataLoader

from torchvision import datasets
import torchvision.transforms as transforms


from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F

import numpy as np

from torchvision.utils import save_image

import matplotlib.pyplot as plt
import math

In [15]:
import os
from os import listdir

In [16]:
BASE_PATH_DATA = '../data/'
#BASE_PATH_DATA = '/scratch/ns3429/sparse-subset/data/'

In [17]:
n_epochs = 100
batch_size = 64
lr = 0.000002
b1 = 0.5
b2 = 0.999
img_size = 28
channels = 1

log_interval = 100


z_size = 40

n = 28 * 28

# from running
# EPSILON = np.finfo(tf.float32.as_numpy_dtype).tiny
#EPSILON = 1.1754944e-38
EPSILON = 1e-10

In [18]:
cuda = True if torch.cuda.is_available() else False

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

device = torch.device("cuda:0" if cuda else "cpu")

In [19]:
print("Device")
print(device)

Device
cuda:0


In [22]:
np.random.seed(100)

In [23]:
import scipy.io as sio

In [24]:
a = sio.loadmat(BASE_PATH_DATA + 'zeisel/CITEseq.mat')
data= a['G'].T
N,d=data.shape
#transformation from integer entries 
data=np.log(data+np.ones(data.shape))
#for i in range(N):
for i in range(d):
    #data[i,:]=data[i,:]/np.linalg.norm(data[i,:])
    data[:,i]= (data[:,i] - np.min(data[:,i])) /  (np.max(data[:,i]) - np.min(data[:, i]))

#load labels from file
a = sio.loadmat(BASE_PATH_DATA + 'zeisel/CITEseq-labels.mat')
l_aux = a['labels']
labels = np.array([i for [i] in l_aux])

#load names from file
a = sio.loadmat(BASE_PATH_DATA + 'zeisel/CITEseq_names.mat')
names=[a['citeseq_names'][i][0][0] for i in range(N)]

In [25]:
slices = np.random.permutation(np.arange(data.shape[0]))
upto = int(.8 * len(data))

train_data = data[slices[:upto]]
test_data = data[slices[upto:]]

train_data = Tensor(train_data).to(device)
test_data = Tensor(test_data).to(device)

In [26]:
print(train_data.std(dim = 0).mean())
print(test_data.std(dim = 0).mean())

tensor(0.1832, device='cuda:0')
tensor(0.1824, device='cuda:0')


In [27]:
def loss_function_per_autoencoder(x, mu_x, logvar_x, mu_latent, logvar_latent):
    # BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    # BCE = F.mse_loss(recon_x, x, reduction='sum')
    loss_rec = -torch.sum(
            (-0.5 * np.log(2.0 * np.pi))
            + (-0.5 * logvar_x)
            + ((-0.5 / torch.exp(logvar_x)) * (x - mu_x) ** 2.0))

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar_latent - mu_latent.pow(2) - logvar_latent.exp())
    #print(loss_rec.item(), KLD.item())
    return loss_rec + 130640 * KLD

In [28]:
# KLD of D(P_1||P_2) where P_i are Gaussians, assuming diagonal
def kld_joint_autoencoders(mu_1, mu_2, logvar_1, logvar_2):
    # equation 6 of Tutorial on Variational Autoencoders by Carl Doersch
    # https://arxiv.org/pdf/1606.05908.pdf
    mu_12 = mu_1 - mu_2
    kld = 0.5 * (-1 - (logvar_1 - logvar_2) + mu_12.pow(2) / logvar_2.exp() + torch.exp(logvar_1 - logvar_2))
    #print(kld.shape)
    kld = torch.sum(kld, dim = 1)
    
    return kld.sum()

In [29]:
# for joint
def loss_function_joint(x, ae_1, ae_2):
    # assuming that both autoencoders return recon_x, mu, and logvar
    # try to make ae_1 the vanilla vae
    # ae_2 should be the L1 penalty VAE
    mu_x_1, logvar_x_1, mu_latent_1, logvar_latent_1 = ae_1(x)
    mu_x_2, logvar_x_2, mu_latent_2, logvar_latent_2 = ae_2(x)
    
    loss_vae_1 = loss_function_per_autoencoder(x, mu_x_1, logvar_x_1, mu_latent_1, logvar_latent_1)
    loss_vae_2 = loss_function_per_autoencoder(x, mu_x_2, logvar_x_2, mu_latent_2, logvar_latent_2)
    joint_kld_loss = kld_joint_autoencoders(mu_latent_1, mu_latent_2, logvar_latent_1, logvar_latent_1)
    #print("Losses")
    #print(loss_vae_1)
    #print(loss_vae_2)
    #print(joint_kld_loss)
    return loss_vae_1, loss_vae_2, joint_kld_loss

Does L1 work if we normalize after every step?

In [30]:
# L1 VAE model we are loading
class VAE_l1_diag(nn.Module):
    def __init__(self, input_size, hidden_layer_size, z_size):
        super(VAE_l1_diag, self).__init__()
        
        self.diag = nn.Parameter(torch.normal(torch.zeros(input_size), 
                                 torch.ones(input_size)).to(device).requires_grad_(True))
        
        # self.fc1 = nn.Linear(input_size, hidden_layer_size)
        self.fc21 = nn.Linear(hidden_layer_size, z_size)
        self.fc22 = nn.Linear(hidden_layer_size, z_size)
        self.fc3 = nn.Linear(z_size, hidden_layer_size)
        self.fc4 = nn.Linear(hidden_layer_size, input_size)
        self.fc5 = nn.Linear(hidden_layer_size, input_size)
        
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_layer_size),
            nn.LeakyReLU(),
            nn.Linear(hidden_layer_size, hidden_layer_size),
            nn.LeakyReLU(),
            nn.Linear(hidden_layer_size, hidden_layer_size),
            nn.LeakyReLU()
        )

    def encode(self, x):
        self.selection_layer = torch.diag(self.diag)
        h0 = torch.mm(x, self.selection_layer)
        h1 = self.encoder(h0)
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h = F.leaky_relu(self.fc3(z))
        mu_x = F.leaky_relu(self.fc4(h))
        #mu_x = self.fc4(h)
        logvar_x = self.fc5(h)
        return mu_x, logvar_x

    def forward(self, x):
        mu_latent, logvar_latent = self.encode(x)
        z = self.reparameterize(mu_latent, logvar_latent)
        mu_x, logvar_x = self.decode(z)
        return mu_x, logvar_x, mu_latent, logvar_latent


In [31]:
def train_l1(df, model, optimizer, epoch):
    model.train()
    train_loss = 0
    permutations = torch.randperm(df.shape[0])
    for i in range(math.ceil(len(df)/batch_size)):
        batch_ind = permutations[i * batch_size : (i+1) * batch_size]
        batch_data = df[batch_ind, :]
        
        optimizer.zero_grad()
        mu_x, logvar_x, mu_latent, logvar_latent = model(batch_data)
        loss = loss_function_per_autoencoder(batch_data, mu_x, logvar_x, mu_latent, logvar_latent)
        loss += 1000000 * torch.norm(model.diag, p = 1)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        with torch.no_grad():
            model.diag.data /= torch.norm(model.diag.data, p = 2)
        
        if i % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(batch_data), len(df),
                100. * i / len(df),
                loss.item() / len(batch_data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(df)))
    
    


In [32]:
def test(df, model, epoch):
    model.eval()
    test_loss = 0
    inds = np.arange(df.shape[0])
    with torch.no_grad():
        for i in range(math.ceil(len(df)/batch_size)):
            batch_ind = inds[i * batch_size : (i+1) * batch_size]
            batch_data = df[batch_ind, :]
            batch_data = batch_data.to(device)
            mu_x, logvar_x, mu_latent, logvar_latent = model(batch_data)
            test_loss += loss_function_per_autoencoder(batch_data, mu_x, logvar_x, mu_latent, logvar_latent).item()


    test_loss /= len(df)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [33]:
model_l1_diag = VAE_l1_diag(500, 200, 50)

model_l1_diag.to(device)
model_l1_optimizer = torch.optim.Adam(model_l1_diag.parameters(), 
                                            lr=lr,
                                            betas = (b1,b2))

In [34]:
for epoch in range(1, n_epochs + 1):
        train_l1(train_data, model_l1_diag, model_l1_optimizer, epoch)
        test(test_data, model_l1_diag, epoch)

Train Epoch: 1 [0/6893 (0%)]	Loss: 6333089.000000
Train Epoch: 1 [6400/6893 (1%)]	Loss: 286486.906250
====> Epoch: 1 Average loss: 343915.2793
====> Test set loss: 10745.4644
Train Epoch: 2 [0/6893 (0%)]	Loss: 286398.000000
Train Epoch: 2 [6400/6893 (1%)]	Loss: 285269.187500
====> Epoch: 2 Average loss: 286553.1171
====> Test set loss: 10155.9909
Train Epoch: 3 [0/6893 (0%)]	Loss: 285184.875000
Train Epoch: 3 [6400/6893 (1%)]	Loss: 284093.093750
====> Epoch: 3 Average loss: 285357.6858
====> Test set loss: 9598.3208
Train Epoch: 4 [0/6893 (0%)]	Loss: 284009.718750
Train Epoch: 4 [6400/6893 (1%)]	Loss: 282927.531250
====> Epoch: 4 Average loss: 284185.8021
====> Test set loss: 9063.6273
Train Epoch: 5 [0/6893 (0%)]	Loss: 282848.343750
Train Epoch: 5 [6400/6893 (1%)]	Loss: 281797.281250
====> Epoch: 5 Average loss: 283036.7262
====> Test set loss: 8552.7886
Train Epoch: 6 [0/6893 (0%)]	Loss: 281705.531250
Train Epoch: 6 [6400/6893 (1%)]	Loss: 280670.906250
====> Epoch: 6 Average loss: 28

Train Epoch: 48 [6400/6893 (1%)]	Loss: 248469.375000
====> Epoch: 48 Average loss: 249394.5546
====> Test set loss: 383.3442
Train Epoch: 49 [0/6893 (0%)]	Loss: 248436.937500
Train Epoch: 49 [6400/6893 (1%)]	Loss: 247925.453125
====> Epoch: 49 Average loss: 248846.0669
====> Test set loss: 378.7061
Train Epoch: 50 [0/6893 (0%)]	Loss: 247887.421875
Train Epoch: 50 [6400/6893 (1%)]	Loss: 247376.828125
====> Epoch: 50 Average loss: 248302.8883
====> Test set loss: 374.0573
Train Epoch: 51 [0/6893 (0%)]	Loss: 247363.109375
Train Epoch: 51 [6400/6893 (1%)]	Loss: 246841.265625
====> Epoch: 51 Average loss: 247766.0136
====> Test set loss: 369.7530
Train Epoch: 52 [0/6893 (0%)]	Loss: 246817.921875
Train Epoch: 52 [6400/6893 (1%)]	Loss: 246320.546875
====> Epoch: 52 Average loss: 247232.5260
====> Test set loss: 366.2332
Train Epoch: 53 [0/6893 (0%)]	Loss: 246290.625000
Train Epoch: 53 [6400/6893 (1%)]	Loss: 245804.578125
====> Epoch: 53 Average loss: 246707.6437
====> Test set loss: 361.4548


Train Epoch: 95 [6400/6893 (1%)]	Loss: 225197.531250
====> Epoch: 95 Average loss: 226004.0299
====> Test set loss: 223.8703
Train Epoch: 96 [0/6893 (0%)]	Loss: 225151.671875
Train Epoch: 96 [6400/6893 (1%)]	Loss: 224724.125000
====> Epoch: 96 Average loss: 225541.6082
====> Test set loss: 222.4652
Train Epoch: 97 [0/6893 (0%)]	Loss: 224692.625000
Train Epoch: 97 [6400/6893 (1%)]	Loss: 224259.109375
====> Epoch: 97 Average loss: 225076.6544
====> Test set loss: 218.2443
Train Epoch: 98 [0/6893 (0%)]	Loss: 224217.187500
Train Epoch: 98 [6400/6893 (1%)]	Loss: 223793.468750
====> Epoch: 98 Average loss: 224610.3708
====> Test set loss: 216.2386
Train Epoch: 99 [0/6893 (0%)]	Loss: 223767.984375
Train Epoch: 99 [6400/6893 (1%)]	Loss: 223329.125000
====> Epoch: 99 Average loss: 224144.4773
====> Test set loss: 214.4231
Train Epoch: 100 [0/6893 (0%)]	Loss: 223287.937500
Train Epoch: 100 [6400/6893 (1%)]	Loss: 222856.218750
====> Epoch: 100 Average loss: 223678.8424
====> Test set loss: 209.20

In [35]:
bins = [10**(-i) for i in range(10)]
bins.reverse()
bins += [10]
np.histogram(model_l1_diag.diag.abs().clone().detach().cpu().numpy(), bins = bins)

(array([  3,  15, 126,  15,   1,   2,  57, 252,  27,   0]),
 array([1.e-09, 1.e-08, 1.e-07, 1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02,
        1.e-01, 1.e+00, 1.e+01]))

In [37]:
with torch.no_grad():
    train_pred = model_l1_diag(train_data[0:64, :])[0]
    train_pred[train_pred < 0.001] = 0 

    test_pred = model_l1_diag(test_data[0:64,:])[0]
    test_pred[test_pred < 0.001] = 0 

In [38]:
test_data[0, :]

tensor([0.5442, 0.6031, 0.4708, 0.5576, 0.3855, 0.5411, 0.5537, 0.4934, 0.5422,
        0.5011, 0.5007, 0.3545, 0.5559, 0.3670, 0.1785, 0.5369, 0.1804, 0.4077,
        0.4889, 0.6549, 0.4487, 0.3518, 0.3118, 0.6650, 0.3306, 0.3359, 0.5101,
        0.3252, 0.5532, 0.1821, 0.3380, 0.4546, 0.2051, 0.5097, 0.5571, 0.3372,
        0.3656, 0.5523, 0.3793, 0.3885, 0.4283, 0.5121, 0.5536, 0.5432, 0.3926,
        0.3274, 0.4116, 0.3199, 0.2978, 0.4827, 0.3418, 0.5646, 0.4051, 0.1440,
        0.3947, 0.2577, 0.4066, 0.3444, 0.3856, 0.4076, 0.3271, 0.4662, 0.1573,
        0.4680, 0.3788, 0.3458, 0.3526, 0.4697, 0.4621, 0.4787, 0.3839, 0.3931,
        0.3947, 0.3900, 0.5435, 0.3632, 0.3662, 0.3182, 0.1832, 0.3874, 0.4308,
        0.0000, 0.5779, 0.3885, 0.3562, 0.0000, 0.2672, 0.4513, 0.4471, 0.1637,
        0.4628, 0.0000, 0.1601, 0.1892, 0.4363, 0.1906, 0.3709, 0.1700, 0.0000,
        0.0000, 0.2522, 0.0000, 0.3841, 0.2080, 0.1686, 0.3274, 0.1560, 0.0000,
        0.0000, 0.4421, 0.0000, 0.2652, 

In [39]:
print(torch.sum(test_pred[0,:] != 0))
print(torch.sum(test_data[0,:] != 0))

tensor(316, device='cuda:0')
tensor(264, device='cuda:0')


In [40]:
with torch.no_grad():
    print(torch.sum((train_pred - train_data[0:64, :]).abs()) / 64/500)
    print(torch.sum((test_pred - test_data[0:64, :]).abs()) / 64/500)

tensor(0.1693, device='cuda:0')
tensor(0.1766, device='cuda:0')


First try Pretrained VAE and then gumble trick with it

Then try joint training VAE and Gumbel Model

In [555]:
# Vanilla VAE model
# try with gaussian decoder
class VAE(nn.Module):
    def __init__(self, input_size, hidden_layer_size, z_size):
        super(VAE, self).__init__()

        #self.fc1 = nn.Linear(input_size, hidden_layer_size)
        self.fc21 = nn.Linear(hidden_layer_size, z_size)
        self.fc22 = nn.Linear(hidden_layer_size, z_size)
        self.fc3 = nn.Linear(z_size, hidden_layer_size)
        self.fc4 = nn.Linear(hidden_layer_size, input_size)
        self.fc5 = nn.Linear(hidden_layer_size, input_size)
        
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_layer_size),
            nn.LeakyReLU(),
            nn.Linear(hidden_layer_size, hidden_layer_size),
            nn.LeakyReLU(),
            nn.Linear(hidden_layer_size, hidden_layer_size),
            nn.LeakyReLU()
        )
        
        #self.decoder = nn.Sequential()

    def encode(self, x):
        #h1 = F.relu(self.fc1(x))
        h1 = self.encoder(x)
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):    
        h = F.leaky_relu(self.fc3(z))
        mu_x = F.leaky_relu(self.fc4(h))
        #mu_x = self.fc4(h)
        logvar_x = self.fc5(h)
        return mu_x, logvar_x

    def forward(self, x):
        mu_latent, logvar_latent = self.encode(x)
        z = self.reparameterize(mu_latent, logvar_latent)
        mu_x, logvar_x = self.decode(z)
        return mu_x, logvar_x, mu_latent, logvar_latent


# Pretrain VAE First

In [453]:
pretrain_vae = VAE(500, 200, 50)

pretrain_vae.to(device)
pretrain_vae_optimizer = torch.optim.Adam(pretrain_vae.parameters(), 
                                            lr=lr,
                                            betas = (b1,b2))

In [454]:
def train(df, model, optimizer, epoch):
    model.train()
    train_loss = 0
    permutations = torch.randperm(df.shape[0])
    for i in range(math.ceil(len(df)/batch_size)):
        batch_ind = permutations[i * batch_size : (i+1) * batch_size]
        batch_data = df[batch_ind, :]
        
        optimizer.zero_grad()
        mu_x, logvar_x, mu_latent, logvar_latent = model(batch_data)
        loss = loss_function_per_autoencoder(batch_data, mu_x, logvar_x, mu_latent, logvar_latent)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if i % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(batch_data), len(df),
                100. * i / len(df),
                loss.item() / len(batch_data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(df)))
    
    


In [455]:
for epoch in range(1, n_epochs + 1):
        train(train_data, pretrain_vae, pretrain_vae_optimizer, epoch)
        test(test_data, pretrain_vae, epoch)

Train Epoch: 1 [0/6893 (0%)]	Loss: 10350.126953
Train Epoch: 1 [6400/6893 (1%)]	Loss: 9280.757812
====> Epoch: 1 Average loss: 9723.0692
====> Test set loss: 9111.0543
Train Epoch: 2 [0/6893 (0%)]	Loss: 9090.483398
Train Epoch: 2 [6400/6893 (1%)]	Loss: 8183.021973
====> Epoch: 2 Average loss: 8584.3976
====> Test set loss: 8067.8318
Train Epoch: 3 [0/6893 (0%)]	Loss: 8080.266602
Train Epoch: 3 [6400/6893 (1%)]	Loss: 7185.016602
====> Epoch: 3 Average loss: 7592.4997
====> Test set loss: 7119.4775
Train Epoch: 4 [0/6893 (0%)]	Loss: 7125.715820
Train Epoch: 4 [6400/6893 (1%)]	Loss: 6290.743164
====> Epoch: 4 Average loss: 6670.1339
====> Test set loss: 6220.7737
Train Epoch: 5 [0/6893 (0%)]	Loss: 6292.028320
Train Epoch: 5 [6400/6893 (1%)]	Loss: 5365.541992
====> Epoch: 5 Average loss: 5783.4901
====> Test set loss: 5342.3373
Train Epoch: 6 [0/6893 (0%)]	Loss: 5352.334961
Train Epoch: 6 [6400/6893 (1%)]	Loss: 4548.109863
====> Epoch: 6 Average loss: 4917.3787
====> Test set loss: 4499.31

Train Epoch: 50 [6400/6893 (1%)]	Loss: 494.331543
====> Epoch: 50 Average loss: 507.5651
====> Test set loss: 506.6712
Train Epoch: 51 [0/6893 (0%)]	Loss: 541.547852
Train Epoch: 51 [6400/6893 (1%)]	Loss: 469.752441
====> Epoch: 51 Average loss: 498.8962
====> Test set loss: 497.8975
Train Epoch: 52 [0/6893 (0%)]	Loss: 478.211456
Train Epoch: 52 [6400/6893 (1%)]	Loss: 475.971069
====> Epoch: 52 Average loss: 490.2321
====> Test set loss: 489.6773
Train Epoch: 53 [0/6893 (0%)]	Loss: 477.898804
Train Epoch: 53 [6400/6893 (1%)]	Loss: 502.425720
====> Epoch: 53 Average loss: 482.2407
====> Test set loss: 481.7344
Train Epoch: 54 [0/6893 (0%)]	Loss: 502.726562
Train Epoch: 54 [6400/6893 (1%)]	Loss: 463.151154
====> Epoch: 54 Average loss: 474.1671
====> Test set loss: 473.3923
Train Epoch: 55 [0/6893 (0%)]	Loss: 471.325439
Train Epoch: 55 [6400/6893 (1%)]	Loss: 481.523254
====> Epoch: 55 Average loss: 466.6507
====> Test set loss: 466.1312
Train Epoch: 56 [0/6893 (0%)]	Loss: 465.814026
Trai

Train Epoch: 100 [6400/6893 (1%)]	Loss: 246.614105
====> Epoch: 100 Average loss: 245.3262
====> Test set loss: 244.7185


In [456]:
with torch.no_grad():
    train_pred = pretrain_vae(train_data[0:64, :])[0]
    train_pred[train_pred < 0.001] = 0 

    test_pred = pretrain_vae(test_data[0:64,:])[0]
    test_pred[test_pred < 0.001] = 0 

In [457]:
with torch.no_grad():
    print(torch.sum((train_pred - train_data[0:64, :]).abs()) / 64/500)
    print(torch.sum((test_pred - test_data[0:64, :]).abs()) / 64/500)

tensor(0.1763, device='cuda:0')
tensor(0.1770, device='cuda:0')


In [458]:
print(torch.sum(test_pred[0,:] != 0))
print(torch.sum(test_data[0,:] != 0))

tensor(329, device='cuda:0')
tensor(214, device='cuda:0')


In [459]:
for p in pretrain_vae.parameters():
    p.requires_grad = False

In [460]:
pretrain_vae.requires_grad_(False)

VAE(
  (fc21): Linear(in_features=200, out_features=50, bias=True)
  (fc22): Linear(in_features=200, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=200, bias=True)
  (fc4): Linear(in_features=200, out_features=500, bias=True)
  (fc5): Linear(in_features=200, out_features=500, bias=True)
  (encoder): Sequential(
    (0): Linear(in_features=500, out_features=200, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=200, out_features=200, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=200, out_features=200, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
  )
)

## Train Gumbel with the Pre-Trained VAE

In [461]:
def train_pre_trained(df, model, optimizer, epoch, pretrained_model):
    model.train()
    train_loss = 0
    permutations = torch.randperm(df.shape[0])
    for i in range(math.ceil(len(df)/batch_size)):
        batch_ind = permutations[i * batch_size : (i+1) * batch_size]
        batch_data = df[batch_ind, :]
        
        optimizer.zero_grad()
        mu_x, logvar_x, mu_latent, logvar_latent = model(batch_data)
        with torch.no_grad():
            _, _, mu_latent_2, logvar_latent_2 = pretrained_model(batch_data)
        
        loss = loss_function_per_autoencoder(batch_data, mu_x, logvar_x, mu_latent, logvar_latent)
        loss += 1000*F.mse_loss(mu_latent, mu_latent_2, reduction = 'sum')
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if i % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(batch_data), len(df),
                100. * i / len(df),
                loss.item() / len(batch_data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(df)))

In [462]:
def gumbel_keys(w):
    # sample some gumbels
    uniform = (1.0 - EPSILON) * torch.rand_like(w) + EPSILON
    z = torch.log(-torch.log(uniform))
    w = w + z
    return w


#equations 3 and 4 and 5
def continuous_topk(w, k, t, separate=False):
    softmax = nn.Softmax(dim = -1)
    khot_list = []
    onehot_approx = torch.zeros_like(w, dtype = torch.float32)
    for i in range(k):
        ### conver the following into pytorch
        #khot_mask = tf.maximum(1.0 - onehot_approx, EPSILON)
        max_mask = 1 - onehot_approx < EPSILON
        khot_mask = 1 - onehot_approx
        khot_mask[max_mask] = EPSILON
        
        w += torch.log(khot_mask)
        #onehot_approx = tf.nn.softmax(w / t, axis=-1)
        onehot_approx = softmax(w/t)
        khot_list.append(onehot_approx)
    if separate:
        return torch.stack(khot_list)
    else:
        return torch.sum(torch.stack(khot_list), dim = 0) 


def sample_subset(w, k, t=0.1):
    '''
    Args:
        w (Tensor): Float Tensor of weights for each element. In gumbel mode
            these are interpreted as log probabilities
        k (int): number of elements in the subset sample
        t (float): temperature of the softmax
    '''
    w = gumbel_keys(w)
    return continuous_topk(w, k, t)

In [463]:
# L1 VAE model we are loading
class VAE_Gumbel(nn.Module):
    def __init__(self, input_size, hidden_layer_size, z_size, k, t = 0.1):
        super(VAE_Gumbel, self).__init__()
        
        self.k = k
        self.t = t
        
        self.weight_creator = nn.Sequential(
            nn.Linear(input_size, hidden_layer_size),
            nn.ReLU(),
            nn.Linear(hidden_layer_size, input_size)
        )
        
        #self.fc1 = nn.Linear(input_size, hidden_layer_size)
        #self.fcextra = nn.Linear(hidden_layer_size, hidden_layer_size)
        self.fc21 = nn.Linear(hidden_layer_size, z_size)
        self.fc22 = nn.Linear(hidden_layer_size, z_size)
        self.fc3 = nn.Linear(z_size, hidden_layer_size)
        self.fc4 = nn.Linear(hidden_layer_size, input_size)
        self.fc5 = nn.Linear(hidden_layer_size, input_size)
        
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_layer_size),
            nn.LeakyReLU(),
            nn.Linear(hidden_layer_size, hidden_layer_size),
            nn.LeakyReLU(),
            nn.Linear(hidden_layer_size, hidden_layer_size),
            nn.LeakyReLU()
        )
        

    def encode(self, x):
        w = self.weight_creator(x)
        subset_indices = sample_subset(w, self.k, self.t)
        x = x * subset_indices
        h1 = self.encoder(x)
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h = F.leaky_relu(self.fc3(z))
        mu_x = F.leaky_relu(self.fc4(h))
        #mu_x = self.fc4(h)
        logvar_x = self.fc5(h)
        return mu_x, logvar_x

    def forward(self, x):
        mu_latent, logvar_latent = self.encode(x)
        z = self.reparameterize(mu_latent, logvar_latent)
        mu_x, logvar_x = self.decode(z)
        return mu_x, logvar_x, mu_latent, logvar_latent


In [464]:
vae_gumbel_with_pre = VAE_Gumbel(500, 200, 50, k = 50)
vae_gumbel_with_pre.to(device)
vae_gumbel_with_pre_optimizer = torch.optim.Adam(vae_gumbel_with_pre.parameters(), 
                                                lr=lr, 
                                                betas = (b1,b2))

In [465]:
for epoch in range(1, n_epochs + 1):
        train_pre_trained(train_data, vae_gumbel_with_pre, vae_gumbel_with_pre_optimizer, epoch, pretrain_vae)
        test(test_data, vae_gumbel_with_pre, epoch)

Train Epoch: 1 [0/6893 (0%)]	Loss: 11354.818359
Train Epoch: 1 [6400/6893 (1%)]	Loss: 10679.607422
====> Epoch: 1 Average loss: 10988.9253
====> Test set loss: 10539.3864
Train Epoch: 2 [0/6893 (0%)]	Loss: 10648.009766
Train Epoch: 2 [6400/6893 (1%)]	Loss: 10089.518555
====> Epoch: 2 Average loss: 10321.4054
====> Test set loss: 9897.3480
Train Epoch: 3 [0/6893 (0%)]	Loss: 9972.563477
Train Epoch: 3 [6400/6893 (1%)]	Loss: 9462.444336
====> Epoch: 3 Average loss: 9690.9522
====> Test set loss: 9291.3823
Train Epoch: 4 [0/6893 (0%)]	Loss: 9395.164062
Train Epoch: 4 [6400/6893 (1%)]	Loss: 8823.656250
====> Epoch: 4 Average loss: 9082.4363
====> Test set loss: 8701.4519
Train Epoch: 5 [0/6893 (0%)]	Loss: 8797.026367
Train Epoch: 5 [6400/6893 (1%)]	Loss: 8240.162109
====> Epoch: 5 Average loss: 8503.0004
====> Test set loss: 8133.9604
Train Epoch: 6 [0/6893 (0%)]	Loss: 8199.973633
Train Epoch: 6 [6400/6893 (1%)]	Loss: 7699.266113
====> Epoch: 6 Average loss: 7927.2575
====> Test set loss: 7

Train Epoch: 50 [6400/6893 (1%)]	Loss: 448.529510
====> Epoch: 50 Average loss: 446.4347
====> Test set loss: 444.3412
Train Epoch: 51 [0/6893 (0%)]	Loss: 441.517548
Train Epoch: 51 [6400/6893 (1%)]	Loss: 452.476532
====> Epoch: 51 Average loss: 439.6078
====> Test set loss: 432.2943
Train Epoch: 52 [0/6893 (0%)]	Loss: 414.686371
Train Epoch: 52 [6400/6893 (1%)]	Loss: 439.610321
====> Epoch: 52 Average loss: 432.2142
====> Test set loss: 426.2065
Train Epoch: 53 [0/6893 (0%)]	Loss: 414.951141
Train Epoch: 53 [6400/6893 (1%)]	Loss: 426.015228
====> Epoch: 53 Average loss: 424.2184
====> Test set loss: 418.5072
Train Epoch: 54 [0/6893 (0%)]	Loss: 404.754425
Train Epoch: 54 [6400/6893 (1%)]	Loss: 418.968933
====> Epoch: 54 Average loss: 417.3463
====> Test set loss: 413.8991
Train Epoch: 55 [0/6893 (0%)]	Loss: 422.851471
Train Epoch: 55 [6400/6893 (1%)]	Loss: 394.760620
====> Epoch: 55 Average loss: 412.2916
====> Test set loss: 407.0802
Train Epoch: 56 [0/6893 (0%)]	Loss: 401.387543
Trai

====> Test set loss: 232.0370
Train Epoch: 100 [0/6893 (0%)]	Loss: 233.685730
Train Epoch: 100 [6400/6893 (1%)]	Loss: 233.150543
====> Epoch: 100 Average loss: 230.6892
====> Test set loss: 228.9556


In [466]:
with torch.no_grad():
    train_pred = vae_gumbel_with_pre(train_data[0:64, :])[0]
    train_pred[train_pred < 0.001] = 0 

    test_pred = vae_gumbel_with_pre(test_data[0:64,:])[0]
    test_pred[test_pred < 0.001] = 0 

In [467]:
with torch.no_grad():
    print(torch.sum((train_pred - train_data[0:64, :]).abs()) / 64/500)
    print(torch.sum((test_pred - test_data[0:64, :]).abs()) / 64/500)

tensor(0.1761, device='cuda:0')
tensor(0.1769, device='cuda:0')


In [468]:
print(torch.sum(test_pred[0,:] != 0))
print(torch.sum(test_data[0,:] != 0))

tensor(312, device='cuda:0')
tensor(300, device='cuda:0')


# Joint Training

In [469]:
# model 1 should be vanilla

def train_joint(df, model1, model2, optimizer, epoch):
    model1.train()
    model2.train()
    train_loss = 0
    permutations = torch.randperm(df.shape[0])
    for i in range(math.ceil(len(df)/batch_size)):
        batch_ind = permutations[i * batch_size : (i+1) * batch_size]
        batch_data = df[batch_ind, :]
        
        optimizer.zero_grad()

        
        loss_vae_1, loss_vae_2, joint_kld_loss = loss_function_joint(batch_data, model1, model2)
        loss = (loss_vae_1 + loss_vae_2 + 1000 * joint_kld_loss)
        loss.backward()
        
        train_loss += loss.item()
        
        optimizer.step()
        
        if i % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(batch_data), len(df),
                100. * i / len(df),
                loss.item() / len(batch_data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(df)))

In [470]:
def test_joint(df, model1, model2, epoch):
    model1.eval()
    model2.eval()
    test_loss = 0
    inds = np.arange(df.shape[0])
    with torch.no_grad():
        for i in range(math.ceil(len(df)/batch_size)):
            batch_ind = inds[i * batch_size : (i+1) * batch_size]
            batch_data = df[batch_ind, :]
            batch_data = batch_data.to(device)
            loss_vae_1, loss_vae_2, joint_kld_loss = loss_function_joint(batch_data, model1, model2)
        
            test_loss += (loss_vae_1 + loss_vae_2 + 1000 * joint_kld_loss).item()


    test_loss /= len(df)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [471]:
joint_vanilla_vae = VAE(500, 200, 50)
joint_vanilla_vae.to(device)

joint_vae_gumbel = VAE_Gumbel(500, 200, 50, k = 50)
joint_vae_gumbel.to(device)


joint_optimizer = torch.optim.Adam(list(joint_vanilla_vae.parameters()) + list(joint_vae_gumbel.parameters()), 
                                                lr=lr, 
                                                betas = (b1,b2))

In [472]:
for epoch in range(1, n_epochs + 1):
    train_joint(train_data, joint_vanilla_vae, joint_vae_gumbel, joint_optimizer, epoch)
    test_joint(test_data, joint_vanilla_vae, joint_vae_gumbel, epoch)

Train Epoch: 1 [0/6893 (0%)]	Loss: 21179.169922
Train Epoch: 1 [6400/6893 (1%)]	Loss: 19550.861328
====> Epoch: 1 Average loss: 20293.5652
====> Test set loss: 19415.2366
Train Epoch: 2 [0/6893 (0%)]	Loss: 19404.109375
Train Epoch: 2 [6400/6893 (1%)]	Loss: 17775.816406
====> Epoch: 2 Average loss: 18564.8208
====> Test set loss: 17725.8562
Train Epoch: 3 [0/6893 (0%)]	Loss: 17721.669922
Train Epoch: 3 [6400/6893 (1%)]	Loss: 16287.279297
====> Epoch: 3 Average loss: 16929.6099
====> Test set loss: 16144.1695
Train Epoch: 4 [0/6893 (0%)]	Loss: 16091.786133
Train Epoch: 4 [6400/6893 (1%)]	Loss: 14760.898438
====> Epoch: 4 Average loss: 15392.7508
====> Test set loss: 14635.8077
Train Epoch: 5 [0/6893 (0%)]	Loss: 14542.001953
Train Epoch: 5 [6400/6893 (1%)]	Loss: 13270.356445
====> Epoch: 5 Average loss: 13870.8954
====> Test set loss: 13077.1411
Train Epoch: 6 [0/6893 (0%)]	Loss: 13196.267578
Train Epoch: 6 [6400/6893 (1%)]	Loss: 11469.669922
====> Epoch: 6 Average loss: 12298.2116
====> 

Train Epoch: 49 [6400/6893 (1%)]	Loss: 978.430786
====> Epoch: 49 Average loss: 961.9789
====> Test set loss: 955.9656
Train Epoch: 50 [0/6893 (0%)]	Loss: 932.983337
Train Epoch: 50 [6400/6893 (1%)]	Loss: 994.602173
====> Epoch: 50 Average loss: 947.0840
====> Test set loss: 939.4949
Train Epoch: 51 [0/6893 (0%)]	Loss: 912.605896
Train Epoch: 51 [6400/6893 (1%)]	Loss: 890.054138
====> Epoch: 51 Average loss: 930.1967
====> Test set loss: 927.2901
Train Epoch: 52 [0/6893 (0%)]	Loss: 965.829590
Train Epoch: 52 [6400/6893 (1%)]	Loss: 894.064453
====> Epoch: 52 Average loss: 915.3140
====> Test set loss: 909.5269
Train Epoch: 53 [0/6893 (0%)]	Loss: 903.015991
Train Epoch: 53 [6400/6893 (1%)]	Loss: 862.359863
====> Epoch: 53 Average loss: 900.1099
====> Test set loss: 895.9617
Train Epoch: 54 [0/6893 (0%)]	Loss: 873.159790
Train Epoch: 54 [6400/6893 (1%)]	Loss: 888.414978
====> Epoch: 54 Average loss: 886.6402
====> Test set loss: 882.5302
Train Epoch: 55 [0/6893 (0%)]	Loss: 885.431763
Trai

====> Test set loss: 488.2371
Train Epoch: 99 [0/6893 (0%)]	Loss: 470.760773
Train Epoch: 99 [6400/6893 (1%)]	Loss: 478.809723
====> Epoch: 99 Average loss: 482.0575
====> Test set loss: 480.1685
Train Epoch: 100 [0/6893 (0%)]	Loss: 477.562256
Train Epoch: 100 [6400/6893 (1%)]	Loss: 465.800781
====> Epoch: 100 Average loss: 476.5967
====> Test set loss: 476.2298


In [473]:
with torch.no_grad():
    train_pred = joint_vae_gumbel(train_data[0:64, :])[0]
    train_pred[train_pred < 0.001] = 0 

    test_pred = joint_vae_gumbel(test_data[0:64,:])[0]
    test_pred[test_pred < 0.001] = 0 

In [474]:
with torch.no_grad():
    print(torch.sum((train_pred - train_data[0:64, :]).abs()) / 64/500)
    print(torch.sum((test_pred - test_data[0:64, :]).abs()) / 64/500)

tensor(0.1751, device='cuda:0')
tensor(0.1734, device='cuda:0')


In [475]:
print(torch.sum(test_pred[0,:] != 0))
print(torch.sum(test_data[0,:] != 0))

tensor(313, device='cuda:0')
tensor(300, device='cuda:0')


### Let's actually Garph this.

### Try it out at Gumbel sparsity of k = 10, 25, 50, 100, 250

### Graph Test MSE Loss

## Graph the mean activations at k = 50

In [476]:
def graph_activations(test_data, model, title, file):
    preds, _, _, _ = model(test_data)
    
    pred_activations = preds.mean(dim = 0)
    
    test_activations = test_data.mean(dim = 0)
    
    x = np.arange(500) + 1
    
    fig = plt.figure()
    plt.plot(x, pred_activations.clone().detach().cpu().numpy(), label = 'Average Predictions')
    plt.plot(x, test_activations.clone().detach().cpu().numpy(), label = 'Average Test Data')
    
    plt.title(title)
    
    plt.legend()
    plt.savefig(file)

In [477]:
graph_activations(test_data, joint_vae_gumbel, 'Joint Gumbel vs Test Means', 
                  '/scratch/ns3429/sparse-subset/joint_gumbel.png')

RuntimeError: CUDA out of memory. Tried to allocate 166.00 MiB (GPU 0; 1.96 GiB total capacity; 836.22 MiB already allocated; 187.69 MiB free; 848.00 MiB reserved in total by PyTorch)

In [ ]:
graph_activations(test_data, vae_gumbel_with_pre, 'Gumbel Matching Pretrained VAE vs Test Means', 
                  '/scratch/ns3429/sparse-subset/pretrained_gumbel.png')

In [ ]:
k_all = [10, 25, 50, 100, 250]

In [152]:
losses_pre = []
losses_joint = []

In [ ]:
for k in k_all:
    vae_gumbel_with_pre = VAE_Gumbel(500, 200, 50, k = k)
    vae_gumbel_with_pre.to(device)
    vae_gumbel_with_pre_optimizer = torch.optim.Adam(vae_gumbel_with_pre.parameters(), 
                                                lr=lr, 
                                                betas = (b1,b2))
    
    joint_vanilla_vae = VAE(500, 200, 50)
    joint_vanilla_vae.to(device)

    joint_vae_gumbel = VAE_Gumbel(500, 200, 50, k = k)
    joint_vae_gumbel.to(device)


    joint_optimizer = torch.optim.Adam(list(joint_vanilla_vae.parameters()) + list(joint_vae_gumbel.parameters()), 
                                                lr=lr, 
                                                betas = (b1,b2))
    
    for epoch in (1, n_epochs + 1):
        train_pre_trained(train_data, vae_gumbel_with_pre, vae_gumbel_with_pre_optimizer, epoch, pretrain_vae)
        train_joint(train_data, joint_vanilla_vae, joint_vae_gumbel, joint_optimizer, epoch)
    
    test_pred_pre = vae_gumbel_with_pre(test_data)[0]
    test_pred_pre[test_pred_pre < 0.001] = 0 
    
    test_pred_joint = joint_vanilla_vae(test_data)[0]
    test_pred_joint[test_pred_joint < 0.001] = 0
    
    with torch.no_grad():
        mae_pre = torch.sum((test_pred_pre - test_data).abs()) / len(test_data) / 500
        mae_joint = torch.sum((test_pred_joint - test_data).abs()) / len(test_data) / 500
    
    losses_pre.append(mae_pre.cpu().item())
    losses_joint.append(mae_joint.cpu().item())
    
fig = plt.figure()
plt.plot(k_all, losses_pre, label = 'Average MAE Losses with Gumbel Matching Pretrained')
plt.plot(k_all, losses_joint, label = 'Average MAE Losses with Gumbel Joint Training')

plt.title("Effect on Sparsity on MAE Loss")
plt.xlabel('Sparsity Level (Number of Non-Zero Features)')
plt.ylabel('Per Neuron Average MAE Loss')
plt.legend()

plt.savefig('/scratch/ns3429/sparse-subset/comparing_across_sparsity.png')